In [5]:

import tensorflow as tf
from tensorflow.keras import layers, Model
import cv2
import pandas as pd
import numpy as np

def load_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32) / 255.0
    return img
train_df = pd.read_csv("COMP90086_2023_TLLdataset/train.csv")
test_df = pd.read_csv('COMP90086_2023_TLLdataset/test_candidates.csv')
train_left_images = [load_image(f"COMP90086_2023_TLLdataset/train/left/{img_name}.jpg") for img_name in train_df['left']]
train_right_images = [load_image(f"COMP90086_2023_TLLdataset/train/right/{img_name}.jpg") for img_name in train_df['right']]


def create_pairs(left_images, right_images):
    pairs = []
    labels = []

    for i in range(len(left_images)):
        # Positive pairing
        pairs.append([left_images[i], right_images[i]])
        labels.append(1)

        # Negative pairing
        random_idx = np.random.randint(0, len(left_images))
        while random_idx == i:  # Ensure it's not the same image
            random_idx = np.random.randint(0, len(left_images))
            
        pairs.append([left_images[i], right_images[random_idx]])
        labels.append(0)

    return np.array(pairs), np.array(labels)

pairs, labels = create_pairs(train_left_images, train_right_images)

# Splitting pairs for training
train_pairs = [pairs[:, 0], pairs[:, 1]]









In [6]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.optimizers import Adam


def cosine_similarity_tensors(vectors):
    x, y = vectors
    x = tf.keras.backend.l2_normalize(x, axis=-1)
    y = tf.keras.backend.l2_normalize(y, axis=-1)
    return tf.keras.backend.sum(x * y, axis=-1, keepdims=True)

def siamese_resnet(input_shape):
    # Define the tensors for the two input images
    left_input = Input(input_shape)
    right_input = Input(input_shape)

    # ResNet50 model (you can set weights='imagenet' if you want to use pretrained weights)
    base_model = ResNet50(weights=None, include_top=False, input_shape=input_shape)
    
    # You can freeze some layers, if required
    for layer in base_model.layers:
        layer.trainable = True

    # Generate the encodings (feature vectors) for the two images
    encoded_left = base_model(left_input)
    encoded_right = base_model(right_input)

    # Flatten and add a dense layer, if required
    encoded_left = Flatten()(encoded_left)
    encoded_right = Flatten()(encoded_right)
    
    # Use cosine similarity to compute the similarity score
    similarity_score = Lambda(cosine_similarity_tensors)([encoded_left, encoded_right])

    # Add a dense layer with sigmoid activation (if you want to scale it between 0 and 1 for binary classification)
    prediction = Dense(1, activation='sigmoid')(similarity_score)

    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[left_input, right_input], outputs=prediction)
    
    return siamese_net

input_shape = (245, 200, 3)  # or whatever your image shape is
model_resnet = siamese_resnet(input_shape)
model_resnet.compile(optimizer=Adam(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy'])


model_resnet.fit(train_pairs, labels, epochs=10, batch_size=16,validation_split=0.1)


Epoch 1/10
225/225 [==============================] - 564s 2s/step - loss: 0.6849 - accuracy: 0.5597 - val_loss: 0.6815 - val_accuracy: 0.5525
Epoch 2/10
225/225 [==============================] - 538s 2s/step - loss: 0.6734 - accuracy: 0.5828 - val_loss: 0.6806 - val_accuracy: 0.5475
Epoch 3/10
225/225 [==============================] - 527s 2s/step - loss: 0.6670 - accuracy: 0.5894 - val_loss: 0.6754 - val_accuracy: 0.5725
Epoch 4/10
225/225 [==============================] - 517s 2s/step - loss: 0.6554 - accuracy: 0.6178 - val_loss: 0.6554 - val_accuracy: 0.6325
Epoch 5/10
225/225 [==============================] - 504s 2s/step - loss: 0.6512 - accuracy: 0.6103 - val_loss: 0.6628 - val_accuracy: 0.6125
Epoch 6/10
 41/225 [====>.........................] - ETA: 6:35 - loss: 0.6443 - accuracy: 0.6402

KeyboardInterrupt: 

In [8]:

def predict_similarity1(left_img, right_img):
    return model_resnet.predict([np.array([left_img]), np.array([right_img])])

predictions = []

# Iterate through each row in the test dataframe
for _, row in test_df.iterrows():
    left_img = load_image(f"COMP90086_2023_TLLdataset/test/left/{row['left']}.jpg")
    confidences = []

    # Predict the similarity score for each candidate "right" image
    for i in range(20): # Assuming there are 20 candidates for each "left" image
        right_img = load_image(f"COMP90086_2023_TLLdataset/test/right/{row[f'c{i}']}.jpg")
        confidences.append(predict_similarity1(left_img, right_img))
    predictions.append(confidences)

# Create a DataFrame to store the predictions
submission_df = pd.DataFrame(predictions, columns=[f'c{i}' for i in range(20)])

submission_df['left'] = test_df['left']

# Rearrange columns to put 'left' at the beginning
cols = ['left'] + [col for col in submission_df if col != 'left']
submission_df = submission_df[cols]
# Write the DataFrame to a CSV file
submission_df.to_csv('kaggle_submission_res_cos.csv', index=False)



1/1 [==============================] - 0s 72ms/step


In [9]:

def extract_value_from_mixed_type(entry):
    # If entry is a numpy array
    if isinstance(entry, (list, np.ndarray)):
        return entry[0][0]
    # If entry is a string
    elif isinstance(entry, str):
        return float(entry.replace('[[', '').replace(']]', ''))
    # If none of the above, return the entry as is (this shouldn't happen, but just in case)
    return entry


for col in submission_df.columns:
    if col != "left":
        submission_df[col] = submission_df[col].apply(extract_value_from_mixed_type)

submission_df.to_csv('kaggle_submission_res_cos.csv', index=False)